In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json("/gardenData.json")

In [ ]:
df_lowes=df[df.retailer =='Lowes']

In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
import pandas as pd
stemmer = SnowballStemmer("english")

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(lemmatize_stemming(token))
            
    return result

In [ ]:
doc_sample = 'This disk has failed many times. I would like to get it replaced.'
print(preprocess(doc_sample))

['disk', 'fail', 'time', 'like', 'replac']


In [ ]:
processed_reviews = []

for i in list(df_lowes.review_text):
    processed_reviews.append(preprocess(i))

In [ ]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in df using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_reviews)

In [ ]:
'''
Create the Bag-of-words model for each review i.e for each review we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(review) for review in processed_reviews]

In [46]:
## for example....
bow_doc_x = bow_corpus[0]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 0 ("bad") appears 1 time.
Word 1 ("constant") appears 1 time.
Word 2 ("day") appears 1 time.
Word 3 ("degre") appears 1 time.
Word 4 ("germin") appears 1 time.
Word 5 ("get") appears 2 time.
Word 6 ("grow") appears 1 time.
Word 7 ("home") appears 1 time.
Word 8 ("miss") appears 1 time.
Word 9 ("open") appears 1 time.
Word 10 ("pick") appears 1 time.
Word 11 ("plant") appears 1 time.
Word 12 ("pound") appears 1 time.
Word 13 ("saw") appears 1 time.
Word 14 ("seed") appears 3 time.
Word 15 ("store") appears 1 time.
Word 16 ("wash") appears 1 time.
Word 17 ("water") appears 1 time.
Word 18 ("weather") appears 1 time.
Word 19 ("week") appears 1 time.


In [47]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10
                                   )

In [48]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.036*"collect" + 0.036*"review" + 0.036*"promot" + 0.028*"plant" + 0.028*"product" + 0.022*"love" + 0.019*"use" + 0.019*"work" + 0.016*"bug" + 0.011*"like"


Topic: 1 
Words: 0.054*"spray" + 0.035*"use" + 0.029*"bug" + 0.026*"easi" + 0.023*"work" + 0.020*"review" + 0.020*"promot" + 0.020*"collect" + 0.019*"plant" + 0.019*"kill"


Topic: 2 
Words: 0.034*"ant" + 0.022*"product" + 0.020*"review" + 0.020*"collect" + 0.020*"promot" + 0.018*"use" + 0.013*"yard" + 0.013*"ironit" + 0.011*"spray" + 0.011*"appli"


Topic: 3 
Words: 0.041*"weed" + 0.019*"grass" + 0.017*"product" + 0.017*"review" + 0.017*"promot" + 0.016*"collect" + 0.016*"area" + 0.015*"tri" + 0.014*"rain" + 0.013*"ive"


Topic: 4 
Words: 0.042*"product" + 0.040*"review" + 0.037*"promot" + 0.037*"collect" + 0.026*"use" + 0.023*"work" + 0.019*"spray" + 0.016*"recommend" + 0.015*"bug" + 0.012*"like"


Topic: 5 
Words: 0.041*"review" + 0.040*"promot" + 0.040*"collect" + 0.037*"insect" + 0.035*"garden" + 0.029*"plan